In [1]:
import meshio
import numpy as np
import pandas as pd
from scipy import ndimage
from netCDF4 import Dataset

# Reading global mesh information

First we read the spherical mesh structure for the considered resolution:

In [24]:
dir_lvl = 'lvl_8'

In [25]:
mesh_struct = np.load(dir_lvl+'/mesh.npz')

print("Field names: {}".format(mesh_struct.files))

ngbIDs = mesh_struct['n']
vertices = mesh_struct['v']
cells = mesh_struct['c']

Field names: ['c', 'n', 'v']


## Paleodata interpolation function

In [26]:
def buildPaleoMesh(time, infile, outfile, rain=False, visvtk=False, filter=2):
    
    paleof = infile+str(time)+"Ma.nc"
    paleom = outfile+str(time)+"Ma"

    paleodata = Dataset(paleof, "r", format="NETCDF4")
    
    if rain:
        paleod = paleodata['z'][:,:].T
    else:
        paleod = np.fliplr(paleodata['z'][:,:].T)
    
    # Apply some smoothing if necessary...
    if filter>0:
        paleod = ndimage.gaussian_filter(paleod,sigma=filter)

    if rain:
        # Interpolate the paleogrid on global mesh
        meshd = ndimage.map_coordinates(paleod, coords , order=2, mode='nearest').astype(np.float)
        # Save the mesh as compressed numpy file for global simulation
        np.savez_compressed(paleom, v=vertices, c=cells, n=ngbIDs.astype(int), r=meshd)
    else:
        # Interpolate the paleogrid on global mesh
        meshd = ndimage.map_coordinates(paleod, coords , order=2, mode='nearest').astype(np.float)
        # Save the mesh as compressed numpy file for global simulation
        np.savez_compressed(paleom, v=vertices, c=cells, n=ngbIDs.astype(int), z=meshd)
        
    print("Processing {} to create {} done".format(paleof,paleom+".npz"))
    
    if visvtk:
        paleovtk = outfile+str(time)+"Ma.vtk"
        if rain:
            vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, point_data={"r":meshd})
        else:
            vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, point_data={"z":meshd})
        meshio.write(paleovtk, vis_mesh)
        print("Writing VTK file {}".format(paleovtk))

    return

## Reading paleodataset

It is assumed that the paleodata files are named using the following convention XXMa.nc where XX is the number of million year before present. 

### Paleo-elevation

We load the corresponding longitudes and latitudes coordinates

In [27]:
mesh_lonlat = np.load(dir_lvl+'/icoord_elev.npz')

print("Field names: {}".format(mesh_lonlat.files))

coords = mesh_lonlat['v']

Field names: ['v']


In [28]:
efile = "data/Scotese-paleomap/"
outfile = dir_lvl+"/elev"

timeframe = 70

buildPaleoMesh(timeframe, efile, outfile, rain=False, visvtk=True)

Processing data/Scotese-paleomap/70Ma.nc to create lvl_8/elev70Ma.npz done
Writing VTK file lvl_8/elev70Ma.vtk


We can do it in a loop like this:

In [29]:
infile = "data/Scotese-paleomap/"
outfile = dir_lvl+"/elev"

# Specify most recent time in Ma
startMa = 0
# Specify deepest time in Ma
endMa = 70
# Specify paleomap interval
dtMa = 5

In [30]:
timeframe = np.flipud(np.arange(startMa,endMa+dtMa,dtMa))

for k in range(len(timeframe)):
    if k == 0:
        buildPaleoMesh(int(timeframe[k]), infile, outfile, rain=False, visvtk=True)
    else:
        buildPaleoMesh(int(timeframe[k]), infile, outfile, rain=False, visvtk=False)

Processing data/Scotese-paleomap/70Ma.nc to create lvl_8/elev70Ma.npz done
Writing VTK file lvl_8/elev70Ma.vtk
Processing data/Scotese-paleomap/65Ma.nc to create lvl_8/elev65Ma.npz done
Processing data/Scotese-paleomap/60Ma.nc to create lvl_8/elev60Ma.npz done
Processing data/Scotese-paleomap/55Ma.nc to create lvl_8/elev55Ma.npz done
Processing data/Scotese-paleomap/50Ma.nc to create lvl_8/elev50Ma.npz done
Processing data/Scotese-paleomap/45Ma.nc to create lvl_8/elev45Ma.npz done
Processing data/Scotese-paleomap/40Ma.nc to create lvl_8/elev40Ma.npz done
Processing data/Scotese-paleomap/35Ma.nc to create lvl_8/elev35Ma.npz done
Processing data/Scotese-paleomap/30Ma.nc to create lvl_8/elev30Ma.npz done
Processing data/Scotese-paleomap/25Ma.nc to create lvl_8/elev25Ma.npz done
Processing data/Scotese-paleomap/20Ma.nc to create lvl_8/elev20Ma.npz done
Processing data/Scotese-paleomap/15Ma.nc to create lvl_8/elev15Ma.npz done
Processing data/Scotese-paleomap/10Ma.nc to create lvl_8/elev10M

### Paleo-precipitation

We can proceed in a similar way to extract the paleo-precipitation dataset...

In [31]:
mesh_lonlat = np.load(dir_lvl+'/icoord_rain.npz')

print("Field names: {}".format(mesh_lonlat.files))

coords = mesh_lonlat['v']

Field names: ['v']


In [32]:
rfile = "data/Scotese-precipitation/"
outfile = dir_lvl+"/rain"

# Specify most recent time in Ma
startMa = 0
# Specify deepest time in Ma
endMa = 70
# Specify paleomap interval
dtMa = 5

In [33]:
timeframe = np.flipud(np.arange(startMa,endMa+dtMa,dtMa))

for k in range(len(timeframe)):
    if k == 0:
        buildPaleoMesh(int(timeframe[k]), rfile, outfile, rain=True, visvtk=True)
    else:
        buildPaleoMesh(int(timeframe[k]), rfile, outfile, rain=True, visvtk=False)

Processing data/Scotese-precipitation/70Ma.nc to create lvl_8/rain70Ma.npz done
Writing VTK file lvl_8/rain70Ma.vtk
Processing data/Scotese-precipitation/65Ma.nc to create lvl_8/rain65Ma.npz done
Processing data/Scotese-precipitation/60Ma.nc to create lvl_8/rain60Ma.npz done
Processing data/Scotese-precipitation/55Ma.nc to create lvl_8/rain55Ma.npz done
Processing data/Scotese-precipitation/50Ma.nc to create lvl_8/rain50Ma.npz done
Processing data/Scotese-precipitation/45Ma.nc to create lvl_8/rain45Ma.npz done
Processing data/Scotese-precipitation/40Ma.nc to create lvl_8/rain40Ma.npz done
Processing data/Scotese-precipitation/35Ma.nc to create lvl_8/rain35Ma.npz done
Processing data/Scotese-precipitation/30Ma.nc to create lvl_8/rain30Ma.npz done
Processing data/Scotese-precipitation/25Ma.nc to create lvl_8/rain25Ma.npz done
Processing data/Scotese-precipitation/20Ma.nc to create lvl_8/rain20Ma.npz done
Processing data/Scotese-precipitation/15Ma.nc to create lvl_8/rain15Ma.npz done
Proc